# A Feasibility Pump heuristic for general mixed-integer problems  논문 구현하기
> Bertacco(2004)


FP는 MIP를 휴리스틱하게 푸는 방법 중 하나이다. 연구를 위해 다음을 구현해봐야해서 정리할 겸 구현해보도록 할 것이다.

$$
\begin{align*}
\qquad Z^* = \min \quad & \mathbf{c}^\top \mathbf{x} \\
\text{s.t.} \quad & \mathbf{A} \mathbf{x} \le \mathbf{b} \\
&{x_j}, \text{ Integer, } \forall j \in \mathcal I.
\end{align*}
$$

- A는 m X n matix이며, I는 정수 조건 변수의 index set이다. 
또한 다음과 같은 변수 bound가 있다고 하자. 

$$
\begin{align*}
l_j \le x_j \le u_j,  \forall j \in \mathcal N.
\end{align*}
$$

- 바운드의 범위는 -Inf~ Inf모두 된다. 
- $\mathcal N$은 모든 변수를 나타낸다. 


MIP문제를 푸는것은 NP-hard문제를 푸는것이라 매우 어려운 문제이다. 휴리스틱으로 푸려는 많은 시도들이 있었고 많은 중요한 결과들을 내놓았다. 
Fischetti, Glover and Lodi (2005)는 general MIP에 대해서 feasible solution을 찾는 FP라는 휴리스틱 도식을 제안하였다. 

$$P:=\{ x : Ax \le b \}$$

- 이는 주어진 MIP의 LP relaxation으로 이루어진 폴리히드론을 나타낸다. 


다음과 같은 notation을 사용할 것이다. 
- $x$ : $x$가 integer
- $\tilde x_j := [x_j]$ : $[ \cdot ]$는 반올림. 

The L1-norm distance 는 다음과 같다.
$$
\begin{align*}
\vartriangle (x, \tilde x) = \sum_{j\in \mathcal I} |x_j-\tilde x_j|
\end{align*}
$$
- continuos variable들은 이 거리계산에 영향을 주지 않는다.

자세하게는 다음과 같다.
$$
\begin{align*}
\vartriangle (x, \tilde x) := \sum_{j\in \mathcal I: \tilde x_j = l_j} (x_j - l_j) + \sum_{j\in \mathcal I: \tilde x_j = u_j} (u_j - x_j) + \sum_{j\in \mathcal I: l_j \le \tilde x_j \le u_j } d_j 
\end{align*}
$$

- $$d_j = |x_j - \tilde x_j|$$

가장 가까운 포인트 $x^*\in P$는 $\tilde x$를 LP solving을 함으로써 쉽게 구해질 수 있다.



In [1]:
# FP
using MathOptInterface, JuMP, Cbc
const MOI = MathOptInterface
using Cbc


m = Model(Cbc.Optimizer)

c = [ -1; -2; -5]
A = [-1  1  3;
      1  3 -7]
b = [-5; 10]
lb = [0;0;0]
ub = [10;Inf;Inf]
index_x = 1:3
index_constraints = 1:2
                       
@variable(m, x[i in index_x], lower_bound = lb[i], upper_bound=ub[i])

var=all_variables(m)
t = Dict(var[2]=>:Int, var[3]=>:Bin)

for i in keys(t)
    if t[i]==:Bin
        JuMP.set_binary(i)
        JuMP.set_lower_bound(i,0)
        JuMP.set_upper_bound(i,1)    
    elseif t[i] == :Int
        JuMP.set_integer(i)
    end
end

for i in keys(t)
    if t[i]==:Bin
        JuMP.unset_binary(i)    
    elseif t[i] == :Int
        JuMP.unset_integer(i)
    end
end

@objective(m, Min, sum( c[i]*var[i] for i in index_x) )
@constraint(m, constraint[j in index_constraints],
               sum( A[j,i]*var[i] for i in index_x ) <= b[j] )

print(m)


Min -x[1] - 2 x[2] - 5 x[3]
Subject to
 constraint[1] : -x[1] + x[2] + 3 x[3] ≤ -5.0
 constraint[2] : x[1] + 3 x[2] - 7 x[3] ≤ 10.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] ≤ 10.0
 x[2] ≤ Inf
 x[3] ≤ 1.0


## Initial 인풋을 위한 계산

In [2]:
# step 1 LP relaxation
JuMP.optimize!(m)
### LP솔루션 저장
solution_k = Dict()
for i in index_x
    solution_k[var[i]] = JuMP.value(var[i])
end

# step 2/3 rounding
### only for integer
solution_k_1 = Dict()
for i in keys(t)
    if solution_k[i]!=round(solution_k[i])
        solution_k_1[i] = round(solution_k[i])
    end
end

# step 6 projection
score = Array{Float64}(undef,0)
for i in keys(solution_k_1)
    if solution_k_1[i] == upper_bound(i)
        push!(score, (upper_bound(i)-solution_k[i]))
    elseif solution_k_1[i] == lower_bound(i)
        push!(score, (solution_k[i])-lower_bound(i))
    else
        push!(score,abs(solution_k[i]-solution_k_1[i])) 
    end
end
dist=sum(score)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
0  Obj 0 Primal inf 1.6666666 (1) Dual inf 12.666666 (3)
0  Obj 0 Primal inf 1.6666666 (1) Dual inf 1e+10 (1)
3  Obj -19.0625
Optimal - objective value -19.0625
Optimal objective -19.0625 - 3 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



0.25000000000000033

In [11]:
maxIter = 100
nIter = 0

# step 4 while
while true
    if dist==0 || nIter > maxIter
        break
    else

        # step 1 LP relaxation
        JuMP.optimize!(m)
        ### LP솔루션 저장
        solution_k = Dict()
        for i in index_x
            solution_k[var[i]] = JuMP.value(var[i])
        end

        # step 2/3 rounding
        ### only for integer
        solution_k_1 = Dict()
        for i in keys(t)
            if solution_k[i]!=round(solution_k[i])
                solution_k_1[i] = round(solution_k[i])
            end
        end

        # step 5 
        
        
        # step 6 projection
        score = Array{Float64}(undef,0)
        for i in keys(solution_k_1)
            if solution_k_1[i] == upper_bound(i)
                push!(score, (upper_bound(i)-solution_k[i]))
            elseif solution_k_1[i] == lower_bound(i)
                push!(score, (solution_k[i])-lower_bound(i))
            else
                push!(score,abs(solution_k[i]-solution_k_1[i])) 
            end
        end


        # step 7 update
        for i in keys(solution_k_1)
            if abs(solution_k[i]-solution_k_1[i])<0
                solution_k[i] = solution_k_1[i]-1
            else
                solution_k[i] = solution_k_1[i]+1
            end
        end

        # step 8 update 2
        for i in keys(solution_k_1)
            if lower_bound(i)<solution_k_1[i]
                JuMP.set_lower_bound(i,solution_k_1[i])
            end
        end

        println(solution_k)
        println("solution_update")
        for i in keys(solution_k_1)
            solution_k[i]=solution_k_1[i]
        end
    end
    println("The total iteration is $nIter ")
    global nIter += 1
    global dist = sum(score)  
    global solution_k
    println("This distance is $dist.")
    println("The solution is ",solution_k) 
    

end

Dict{Any,Any}(x[2]=>3.0,x[1]=>10.0,x[3]=>2.0)
solution_update
The total iteration is 0 
This distance is 0.2500000000000001.
The solution is Dict{Any,Any}(x[2]=>2.0,x[1]=>10.0,x[3]=>1.0)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
Primal infeasible - objective value -19.0625
PrimalInfeasible objective -19.0625 - 0 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0
Total iterations:           0
Time (CPU seconds):         0.00
Time (Wallclock Seconds):   0.00

Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Dict{Any,Any}(x[2]=>2.0,x[1]=>10.0,x[3]=>1.0)
solution_update
The total iteration is 1 
This distance is 0.0.
The solution is Dict{Any,Any}(x[2]=>2.0,x[1]=>10.0,x[3]=>1.0)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Inter

## The result is followed

In [5]:
println("Optimal solutions:")
println("x1 = ", JuMP.value(var[1]))
println("x2 = ", JuMP.value(var[2]))
println("x3 = ", JuMP.value(var[3]))

println("The objective vablue = ", JuMP.objective_value(m))

Optimal solutions:
x1 = 10.0
x2 = 2.0
x3 = 1.0
The objective vablue = -19.0


## using Cbc

In [4]:
m = Model(Cbc.Optimizer)

c = [ -1; -2; -5]
A = [-1  1  3;
      1  3 -7]
b = [-5; 10]
lb = [0;0;0]
ub = [10;Inf;Inf]
index_x = 1:3
index_constraints = 1:2
                       
@variable(m, x[i in index_x], lower_bound = lb[i], upper_bound=ub[i])

var=all_variables(m)
t = Dict(var[2]=>:Int, var[3]=>:Bin)

for i in keys(t)
    if t[i]==:Bin
        JuMP.set_binary(i)
        JuMP.set_lower_bound(i,0)
        JuMP.set_upper_bound(i,1)    
    elseif t[i] == :Int
        JuMP.set_integer(i)
    end
end

@objective(m, Min, sum( c[i]*var[i] for i in index_x) )
@constraint(m, constraint[j in index_constraints],
               sum( A[j,i]*var[i] for i in index_x ) <= b[j] )

print(m)

JuMP.optimize!(m)
        
println("Optimal solutions:")
println("x1 = ", JuMP.value(var[1]))
println("x2 = ", JuMP.value(var[2]))
println("x3 = ", JuMP.value(var[3]))

println("The objective vablue = ", JuMP.objective_value(m))

Min -x[1] - 2 x[2] - 5 x[3]
Subject to
 constraint[1] : -x[1] + x[2] + 3 x[3] ≤ -5.0
 constraint[2] : x[1] + 3 x[2] - 7 x[3] ≤ 10.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] ≤ 10.0
 x[2] ≤ Inf
 x[3] ≤ 1.0
 x[2] integer
 x[3] binary
Optimal solutions:
x1 = 10.0
x2 = 2.0
x3 = 1.0
The objective vablue = -19.0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is -19.0625 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 2 rows, 3 columns (3 integer (1 of which binary)) and 6 elements
Cbc0012I Integer solution of -19 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -19, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -19 to -19
Probing was tried 0 